**Copyright 2021 The TensorFlow Hub Authors.**

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2021 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/hub/tutorials/senteval_for_universal_sentence_encoder_cmlm"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/hub/tutorials/senteval_for_universal_sentence_encoder_cmlm.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/hub/tutorials/senteval_for_universal_sentence_encoder_cmlm.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/hub/tutorials/senteval_for_universal_sentence_encoder_cmlm.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载此 notebook</a></td>
  <td>     <a href="https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1"><img src="https://tensorflow.google.cn/images/hub_logo_32px.png">查看 TF Hub 模型 </a> </td>
</table>

# Universal Sentence Encoder SentEval 演示

本 Colab 使用 [SentEval](https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1) 工具套件演示 [Universal Sentence Encoder CMLM 模型](https://github.com/facebookresearch/SentEval)，该工具套件是用于测量句子嵌入质量的库。SentEval 工具套件包括一组多样化的下游任务，能够评估嵌入模型的泛化能力并评估编码的语言属性。

运行前两个代码块设置环境，在第三个代码块中，可以选择一个 SentEval 任务来评估模型。建议使用 GPU 运行时来运行本 Colab。

要了解有关 Universal Sentence Encoder CMLM 模型的更多信息，请参阅 https://openreview.net/forum?id=WDVD4lUCTzU。

In [ ]:
#@title Install dependencies
!pip install --quiet "tensorflow-text==2.11.*"
!pip install --quiet torch==1.8.1

## 下载 SentEval 和任务数据

本步骤从 github 下载 SentEval 并执行数据脚本下载任务数据。可能需要长达 5 分钟的时间才能完成。

In [ ]:
#@title Install SentEval and download task data
!rm -rf ./SentEval
!git clone https://github.com/facebookresearch/SentEval.git
!cd $PWD/SentEval/data/downstream && bash get_transfer_data.bash > /dev/null 2>&1

#执行 SentEval 评估任务 以下代码块执行 SentEval 任务并输出结果，选择以下任务之一来评估 USE CMLM 模型：

```
MR	CR	SUBJ	MPQA	SST	TREC	MRPC	SICK-E
```

选择要运行的模型、参数和任务。可以使用 rapid prototyping 参数减少计算时间以更快获得结果。

使用 **'rapid prototyping'** 参数完成任务通常需要 5-15 分钟，使用 **'slower, best performance'** 参数最多需要一个小时。

```
params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
params['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                 'tenacity': 3, 'epoch_size': 2}
```

要获得更好的结果，请使用较慢的 **'slower, best performance'** 参数，计算时间可能长达 1 小时：

```
params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
params['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 16,
                                 'tenacity': 5, 'epoch_size': 6}
```


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(f'{os.getcwd()}/SentEval')

import tensorflow as tf

# Prevent TF from claiming all GPU memory so there is some left for pytorch.
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Memory growth needs to be the same across GPUs.
  for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

import tensorflow_hub as hub
import tensorflow_text
import senteval
import time

PATH_TO_DATA = f'{os.getcwd()}/SentEval/data'
MODEL = 'https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1' #@param ['https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1', 'https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-large/1']
PARAMS = 'rapid prototyping' #@param ['slower, best performance', 'rapid prototyping']
TASK = 'CR' #@param ['CR','MR', 'MPQA', 'MRPC', 'SICKEntailment', 'SNLI', 'SST2', 'SUBJ', 'TREC']

params_prototyping = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
params_prototyping['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                 'tenacity': 3, 'epoch_size': 2}

params_best = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
params_best['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 16,
                                 'tenacity': 5, 'epoch_size': 6}

params = params_best if PARAMS == 'slower, best performance' else params_prototyping

preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1")

inputs = tf.keras.Input(shape=tf.shape(''), dtype=tf.string)
outputs = encoder(preprocessor(inputs))

model = tf.keras.Model(inputs=inputs, outputs=outputs)

def prepare(params, samples):
    return

def batcher(_, batch):
    batch = [' '.join(sent) if sent else '.' for sent in batch]
    return model.predict(tf.constant(batch))["default"]


se = senteval.engine.SE(params, batcher, prepare)
print("Evaluating task %s with %s parameters" % (TASK, PARAMS))
start = time.time()
results = se.eval(TASK)
end = time.time()
print('Time took on task %s : %.1f. seconds' % (TASK, end - start))
print(results)


#了解更多

- 在 [TensorFlow Hub](https://tfhub.dev) 上查找更多文本嵌入模型
- 另请参阅 [多语言 Universal Sentence Encoder CMLM 模型](https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base-br/1)
- 查看其他 [Universal Sentence Encoder 模型](https://tfhub.dev/google/collections/universal-sentence-encoder/1)

## 参考

- Ziyi Yang, Yinfei Yang, Daniel Cer, Jax Law, Eric Darve. [Universal Sentence Representations Learning with Conditional Masked Language Model. November 2020](https://openreview.net/forum?id=WDVD4lUCTzU)
